In [1]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Milliman PAC File ETL
1) log into SFTP site: https://secureftp.milliman.com  
2) In the "from milliman" folder, go to the latest directory  
     - In this case, the latest directory is "20221216"  
3) Make sure you have access to the following folder: \\Main Street Health\MSH_Strategy_Group - ACO Analyses\Raw MIlliman Flat FIles\  
     - create folder with same datestamp as file from milliman, in this case "20221216"  
4) Download the files from the SFTP site  
     - in this case there are 3  
5) copy the data disctionary to the folder on the network drive.  
6) download the prospective and retrospective CSV files into database tables in both the MSH and CB datagrip environments (Alan has a place he puts them)  
7) delete the last 30 columns from these database tables (we will create a "pivot file" with these 30 columns):  
8) with these 30 columns, run Alan's "pivot code" to create these files:  
     - strategic_milliman_prosp_20221128_pivot
     -  strategic_milliman_retro_20221128_pivot
9) copy 4 files to the "20221216" folder  
     - prospective flat file (without the 30 MA columns)
     - retrospective flat file (without the 30 MA columns)
     - prospective MA pivot file
     - retrospective MA pivor file
10) email Ali and Ben that process is complete with the following summaries that you can pull via queries and paste in excel into the same folder.  

### Config

In [2]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20240517'
pro_file_name = 'ACO Builder MSSP Prospective Explorer v2025.2.txt'
retro_file_name = 'ACO Builder MSSP Retrospective Explorer v2025.2.txt'

### Pro load

In [3]:
pro_df = pd.read_csv(f'{data_dir}/{current_folder}/{pro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
pro_df = cb_utils.df_format_columns(pro_df)
print(f'{pro_df.shape[0]} rows, {pro_df.shape[1]} columns')
pro_df.head()

1073472 rows, 61 columns


,pac_id,deid_tin,year,program_type,provider_name,mssp_aco_name,reach_aco_name,practice_state,practice_msa,practice_st_msa,practice_msa_name,practice_zip,pys_esrd,pys_dis,pys_ad,pys_and,pys_total,rs_esrd_legacy,rs_dis_legacy,rs_ad_legacy,rs_and_legacy,risk_composite_legacy,rs_esrd_v24,rs_dis_v28,rs_ad_v28,rs_and_v28,risk_composite_v28,rs_esrd_demog,rs_dis_demog,rs_ad_demog,rs_and_demog,risk_composite_demog,paid_esrd,paid_dis,paid_ad,paid_and,paid_composite,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_composite,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_composite,regional_efficiency_pro,reg_ret_exp_esrd_v24,reg_ret_exp_dis_v28,reg_ret_exp_ad_v28,reg_ret_exp_and_v28,reg_ret_exp_composite_v28,regional_efficiency_ret_v28,reg_pro_exp_esrd_v24,reg_pro_exp_dis_v28,reg_pro_exp_ad_v28,reg_pro_exp_and_v28,reg_pro_exp_composite_v28,regional_efficiency_pro_v28
0,42100091,441836,2016,Prospective,HARTSVILLE PRIMARY CARE CENTER,NC TN MSSP CHC,NaN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",37074.0,0.0,5.00,0.00,2.00,7.00,NaN,0.863795,NaN,0.739380,0.859959,NaN,0.863795,NaN,0.739380,0.859959,NaN,1.001509,NaN,1.119004,1.035079,NaN,7121.444000,NaN,566.425000,5248.581429,0.0,10157.99,0.00,9727.06,9677.555738,0.630665,0.0,9646.00,0.00,9412.85,9233.030572,0.661028,0.0,10157.99,0.00,9727.06,9677.555738,0.630665,0.0,9646.00,0.00,9412.85,9233.030572,0.661028
1,42100091,441836,2017,Prospective,HARTSVILLE PRIMARY CARE CENTER,NC TN MSSP CHC,NaN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",37074.0,0.0,7.08,0.92,6.00,14.00,NaN,0.932630,0.752984,0.750412,0.854342,NaN,0.822170,0.945082,0.817481,0.866952,NaN,1.120542,0.871022,1.178063,1.128797,NaN,8765.182203,43792.967391,1242.408333,7842.962143,0.0,11900.28,17513.31,10026.15,11358.163673,0.808240,0.0,10674.83,16837.99,9594.32,10479.976924,0.875968,0.0,11288.74,17598.56,9972.35,10704.671486,0.845107,0.0,10168.06,16995.97,9535.23,9947.387910,0.909444
2,42100091,441836,2018,Prospective,HARTSVILLE PRIMARY CARE CENTER,NC TN MSSP CHC,NaN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",37074.0,0.0,7.00,1.00,9.00,17.00,NaN,0.755567,2.139504,0.746941,0.779248,NaN,0.749534,1.382706,0.826464,0.773139,NaN,1.119854,0.749096,1.051986,1.062115,NaN,24440.150000,3757.030000,3642.783333,12213.125294,0.0,10128.24,17690.27,10204.42,12079.163437,1.297521,0.0,10591.36,17856.92,9851.13,12111.698118,1.294035,0.0,10134.85,17773.01,10255.81,11719.540343,1.347902,0.0,10759.92,17954.16,9946.30,11812.961388,1.337243
3,42100091,441836,2019,Prospective,HARTSVILLE PRIMARY CARE CENTER,NC TN MSSP CHC,NaN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",37074.0,0.0,7.58,1.00,7.42,16.00,NaN,1.442499,1.348140,0.846099,1.127700,NaN,1.208815,1.449134,0.828398,1.043732,NaN,0.992623,0.907614,1.147268,1.059026,NaN,7320.637203,17721.800000,10229.787062,9319.828125,0.0,11657.38,18373.78,10513.03,12095.152494,0.683287,0.0,11434.88,18332.67,10028.94,11788.809232,0.701042,0.0,11078.98,18365.63,10554.33,11557.284513,0.772615,0.0,10995.70,18384.85,10144.58,11362.440375,0.785864
4,42100091,441836,2020,Prospective,HARTSVILLE PRIMARY CARE CENTER,NC TN MSSP CHC,NaN,Tennessee,34980.0,TN-34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",37074.0,0.0,10.00,2.33,7.42,19.75,NaN,0.736111,1.214575,1.090537,0.908417,NaN,0.714473,1.291778,1.138687,0.921810,NaN,0.960056,0.986459,1.116323,1.021880,NaN,17862.057000,30592.888412,6626.024259,15142.637975,0.0,10737.27,20318.36,9180.01,11750.644964,1.418583,0.0,10666.57,19361.55,8642.32,11328.208114,1.471483,0.0,9688.76,20174.22,9160.97,11389.061079,1.442355,0.0,9595.68,19342.80,8664.51,10984.678151,1.495453


In [4]:
conn = cb_utils.get_engine(source='msh_analytics')
pro_table_name = f'milliman_pro_{current_folder}'
pro_schema = 'raw
print(f'select * from {pro_schema}.{pro_table_name}')
# pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

1073472

In [ ]:
pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

In [ ]:
ret_df = pd.read_csv(f'{data_dir}/{current_folder}/{retro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
ret_df = cb_utils.df_format_columns(ret_df)
print(f'{ret_df.shape[0]} rows, {ret_df.shape[1]} columns')
ret_df.head()

In [ ]:
ret_table_name = f'milliman_ret_{current_folder}'
ret_schema = 'raw
print(f'select * from {ret_schema}.{ret_table_name}')

In [6]:
ret_df.to_sql(f'milliman_ret_{current_folder}', conn, schema='raw', index=False, method='multi', chunksize=1000) # if_exists='replace', 

1073472

In [9]:
ret_df.year.value_counts()

2016    93392
2017    90403
2018    87384
2019    84748
2020    82510
2021    79560
2022    76027
2023    75310
Name: year, dtype: int64

In [10]:
for yr in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    ret_df.loc[ret_df.year == yr].to_csv(f'{data_dir}/{current_folder}/ret_{yr}.csv',index=False)

# Did not use below

In [6]:
conn = cb_utils.get_engine()
pro_df.to_sql(f'milliman_pro_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # , if_exists='replace'

613556

### Retro load

In [33]:

ret_df.to_csv('ret.csv',index=False)

In [10]:
conn = cb_utils.get_engine()
ret_df.to_sql(f'milliman_ret_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # if_exists='replace', 

594838

### Pivots

In [11]:
def pivot_ma_plan_enrollment(df):
    dfs = []
    for i in range(1, 16):
        name, enrollment = f'ma_plan_{i}_name', f'ma_plan_{i}_enrollment'
        plan = df[['deid_tin', 'pac_id', 'year', name, enrollment]]
        plan = plan.loc[~(plan[name].isna()) & ~(plan[enrollment].isna())]
        plan.columns = ['deid_tin', 'pac_id', 'year', 'ma_plan_name', 'ma_enrollment']
        dfs.append(plan)
    return pd.concat(dfs)

In [12]:
ret_pivot_df = pivot_ma_plan_enrollment(ret_df)

KeyError: "['ma_plan_1_name', 'ma_plan_1_enrollment'] not in index"

In [ ]:
ret_pivot_df.head()

In [13]:
pro_pivot_df = pivot_ma_plan_enrollment(pro_df)

KeyError: "['ma_plan_1_name', 'ma_plan_1_enrollment'] not in index"

In [ ]:
pro_pivot_df.head()

### Summaries 
### Prospective

In [ ]:
s1 = ret_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s1.head()

### Retro

In [ ]:
s2 = pro_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s2.head()

### Prospective

In [ ]:
s3 = pro_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s3.head()

### Retro

In [ ]:
s4 = ret_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s4.head()

### Outputs

In [ ]:
ret_df.loc[:, ret_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_ret_{current_folder}.csv', index=False)

In [ ]:
pro_df.loc[:, pro_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_pro_{current_folder}.csv', index=False)

In [ ]:
pro_pivot_df.to_csv(f'{data_dir}/{current_folder}/pro_pivot.csv', index=False)
ret_pivot_df.to_csv(f'{data_dir}/{current_folder}/ret_pivot.csv', index=False)

In [ ]:
with pd.ExcelWriter(f'{data_dir}/{current_folder}/summary.xlsx') as writer:
    s1.to_excel(writer, sheet_name='retrospective', index=False)
    s2.to_excel(writer, sheet_name='prospective', index=False)
    s3.to_excel(writer, sheet_name='prospective_pivot', index=False)
    s4.to_excel(writer, sheet_name='retrospective_pivot', index=False)